In [1]:
import torch
import lightning as L
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import BatchSizeFinder, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger
import neptune
import os

### Loading and PreProcessing the Data


txt to csv if needed

In [ ]:
# import csv

# name_open = 'Data.txt'
# name_new = 'datatest2.csv'
# delimiter = '|'  # Adjust this if your actual delimiter is different

# # Read from text file and write to CSV file
# with open(name_open, 'r') as f:
#     # Read the lines
#     lines = f.readlines()

# # Extract headers from the first line
# headers = [header.strip() for header in lines[0].split(delimiter)]

# # Open a new CSV file to write the data
# with open(name_new, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile, escapechar='\\')  # Specify escapechar to handle special characters
    
#     # Write headers
#     writer.writerow(headers)

#     # Write rows
#     for line in lines[1:]:
#         row = [item.strip() for item in line.split(delimiter)]
#         writer.writerow(row)

# print(f"Conversion completed. Data written to {name_new}")


In [2]:
#relevent columes

relevent_features = ['_totl,_time','Vtrue,_ktgs','pitch,__deg','_roll,__deg','hding,_true','__lat,__deg','__lon,__deg','___CG,ftMSL','____X,____m','____Y,____m','____Z,____m','___vX,__m/s','___vY,__m/s','___vZ,__m/s'
                     ,'_elev,stick','ailrn,stick','ruddr,stick','thro1,engin'
]

train_data = pd.read_csv('data/Data2.csv', skiprows=1)
train_data = train_data.dropna(axis=1, how='all')
train_data = train_data[relevent_features]
train_data

val_data = pd.read_csv('data/Data3.csv', skiprows=1)
val_data = val_data.dropna(axis=1, how='all')
val_data = val_data[relevent_features]
val_data

test_data = pd.read_csv('data/Data4.csv', skiprows=1)
test_data = train_data.dropna(axis=1, how='all')
test_data = train_data[relevent_features]
test_data

,"_totl,_time","Vtrue,_ktgs","pitch,__deg","_roll,__deg","hding,_true","__lat,__deg","__lon,__deg","___CG,ftMSL","____X,____m","____Y,____m","____Z,____m","___vX,__m/s","___vY,__m/s","___vZ,__m/s","_elev,stick","ailrn,stick","ruddr,stick","thro1,engin"
0,0.05025,0.08962,2.50369,0.02244,92.61546,45.72394,-122.55013,292.82214,35021.74609,-56.05212,-24987.36914,-0.01215,0.05113,0.00660,0.00000,0.00000,0.00000,0.0
1,0.14557,0.13320,2.78831,0.09394,92.62069,45.72394,-122.55013,292.83615,35021.74219,-56.04783,-24987.36719,-0.06101,0.03080,0.01416,0.00000,0.00000,0.00000,0.0
2,0.24257,0.15175,3.15759,0.17618,92.62554,45.72394,-122.55013,292.83878,35021.73438,-56.04699,-24987.36719,-0.07731,-0.00651,0.01524,0.00000,0.00000,0.00000,0.0
3,0.32532,0.15559,3.47054,0.23833,92.62866,45.72394,-122.55013,292.83459,35021.72656,-56.04823,-24987.36523,-0.07660,-0.01822,0.01317,0.00000,0.00000,0.00000,0.0
4,0.41103,0.15037,3.77623,0.29061,92.62982,45.72394,-122.55013,292.82932,35021.72266,-56.04979,-24987.36523,-0.07464,-0.01758,0.00869,0.00000,0.00000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,677.01807,102.64950,5.03510,-15.52756,92.21129,46.76450,-122.64340,10524.41309,27253.54688,3081.13818,-29478.54102,52.76324,2.18193,2.32113,0.01481,-0.03281,-0.00584,1.0
7849,677.11859,102.61120,5.03769,-15.75969,91.90834,46.76450,-122.64333,10525.20117,27258.84961,3081.35693,-29478.32422,52.75458,2.17173,2.05556,0.01505,-0.03179,-0.00577,1.0
7850,677.21912,102.57314,5.04044,-15.98804,91.60071,46.76449,-122.64326,10525.98730,27264.15039,3081.57471,-29478.13281,52.74484,2.16175,1.78573,0.01529,-0.03097,-0.00570,1.0
7851,677.31958,102.53516,5.04336,-16.21258,91.28848,46.76449,-122.64320,10526.76953,27269.45117,3081.79126,-29477.96875,52.73388,2.15198,1.51168,0.01552,-0.02800,-0.00564,1.0


# CREATING A MODEL

In [3]:
class FlightDataset(Dataset):
    def __init__(self, df, seq_len):
        # Keeping only the relevant features
        relevant_features = ['_totl,_time', 'Vtrue,_ktgs', 'pitch,__deg', '_roll,__deg', 'hding,_true', '__lat,__deg', '__lon,__deg', '___CG,ftMSL', '____X,____m', '____Y,____m', '____Z,____m', '___vX,__m/s', '___vY,__m/s', '___vZ,__m/s', '_elev,stick', 'ailrn,stick', 'ruddr,stick', 'thro1,engin']
        
        self.df = df[relevant_features].copy()  # Use copy to avoid modifying the original DataFrame

        # Changing the names of the features for easier use
        self.df.columns = ['time', 'vt', 'pitch', 'roll', 'hding', 'lat', 'lon', 'alt', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'elev_stick', 'ailrn_stick', 'ruddr_stick', 'throttle']
        
        # Instantiate the StandardScaler
        self.scaler = StandardScaler()
        
        # Fit and transform the data
        self.df = pd.DataFrame(self.scaler.fit_transform(self.df), columns=self.df.columns)
        
        # Define sequence length
        self.seq_len = seq_len

    def __len__(self):
        # Making the length shorter by seq_len so we always have a next_state
        return len(self.df) - self.seq_len

    def __getitem__(self, idx):
        # Current state is the sequence of states from idx to idx + seq_len
        current_state = self.df.iloc[idx:idx + self.seq_len].values.astype(float)
        # Next state is the state at idx + seq_len
        next_state = self.df.iloc[idx + self.seq_len][['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']].values.astype(float)

        return torch.tensor(current_state, dtype=torch.float32), torch.tensor(next_state, dtype=torch.float32)
    
    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)


In [4]:
class FlightLSTM(pl.LightningModule):
    def __init__(self,input_dim,hidden_dim,num_layers,output_dim,lr=0.001):
        super(FlightLSTM,self).__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.lr = lr
        self.criterion = nn.MSELoss(reduction='none')

        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []
        
    def forward(self,x):
        out,_ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out
    
    def compute_metrics(self, y_true, y_pred):
        mse = np.mean((y_true - y_pred) ** 2, axis=0)
        mae = np.mean(np.abs(y_true - y_pred), axis=0)
        return mse, mae
    
    def training_step(self,batch,batch_idx):
        x, y = batch
        output = self(x)
        loss = self.criterion(output, y).mean()
        self.log("train/batch/loss", loss, prog_bar=False)

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"train/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"train/batch/mae_{feature}", mae[i], prog_bar=False)

        self.training_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}
    
    def on_train_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.training_step_outputs])
        mse = np.vstack([output["mse"] for output in self.training_step_outputs])
        mae = np.vstack([output["mae"] for output in self.training_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.training_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.training_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("train/epoch/loss", loss.mean())  # Log training epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"train/epoch/mse_{feature}", mse_all[i])  # Log training epoch MSE for each feature
            self.log(f"train/epoch/mae_{feature}", mae_all[i])  # Log training epoch MAE for each feature
        self.log("train/epoch/r2", r2_all)  # Log training epoch R²

        print("Training Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print
    

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = self.criterion(output, y).mean()

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"val/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"val/batch/mae_{feature}", mae[i], prog_bar=False)

        self.validation_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}
    
    def on_validation_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.validation_step_outputs])
        mse = np.vstack([output["mse"] for output in self.validation_step_outputs])
        mae = np.vstack([output["mae"] for output in self.validation_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.validation_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.validation_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("val/epoch/loss", loss.mean())  # Log validation epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"val/epoch/mse_{feature}", mse_all[i])  # Log validation epoch MSE for each feature
            self.log(f"val/epoch/mae_{feature}", mae_all[i])  # Log validation epoch MAE for each feature
        self.log("val/epoch/r2", r2_all)  # Log validation epoch R²

        print("Validation Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.validation_step_outputs.clear()  # free memory

    def test_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = self.criterion(output, y).mean()

        y_true = y.cpu().detach().numpy()
        y_pred = output.cpu().detach().numpy()

        mse, mae = self.compute_metrics(y_true, y_pred)

        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"test/batch/mse_{feature}", mse[i], prog_bar=False)
            self.log(f"test/batch/mae_{feature}", mae[i], prog_bar=False)

        self.test_step_outputs.append({"loss": loss, "mse": mse, "mae": mae, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "mse": mse, "mae": mae}

    def on_test_epoch_end(self):
        loss = np.array([output["loss"].detach().cpu().numpy() for output in self.test_step_outputs])
        mse = np.vstack([output["mse"] for output in self.test_step_outputs])
        mae = np.vstack([output["mae"] for output in self.test_step_outputs])

        y_true_all = np.vstack([output["y_true"] for output in self.test_step_outputs])
        y_pred_all = np.vstack([output["y_pred"] for output in self.test_step_outputs])

        mse_all = np.mean(mse, axis=0)
        mae_all = np.mean(mae, axis=0)
        r2_all = r2_score(y_true_all, y_pred_all)

        self.log("test/epoch/loss", loss.mean())  # Log test epoch loss
        for i, feature in enumerate(['lat', 'lon', 'alt', 'pitch', 'roll', 'hding', 'vx', 'vy', 'vz']):
            self.log(f"test/epoch/mse_{feature}", mse_all[i])  # Log test epoch MSE for each feature
            self.log(f"test/epoch/mae_{feature}", mae_all[i])  # Log test epoch MAE for each feature
        self.log("test/epoch/r2", r2_all)  # Log test epoch R²

        print("Test Epoch End: Loss:", loss.mean(), "MSE:", mse_all, "MAE:", mae_all, "R²:", r2_all)  # Debugging print

        self.test_step_outputs.clear()  # free memory
        
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    



In [5]:
def seed_everything(seed=42):
    # random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    pl.seed_everything(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
seed_everything()

Seed set to 42


In [6]:
sequance_length = 50
train_dataset = FlightDataset(train_data, sequance_length)
val_dataset = FlightDataset(val_data, sequance_length)
test_dataset = FlightDataset(test_data, sequance_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YTk3YWUxOS1lOWY2LTQ4NzUtOTMzMC00MDExYzA0N2UwNzAifQ==",
    project="peridan/PINN1",
    tags=["notebook"],
    log_model_checkpoints=False
)

In [8]:
seed_everything()
input_dim = len(train_data.columns)
hidden_dim = 128
num_layers = 2
output_dim = 9
model = FlightLSTM(input_dim, hidden_dim, num_layers, output_dim)
trainer = pl.Trainer(
    logger = neptune_logger,
    max_epochs=10
    )
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)
neptune_logger.experiment.stop()


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/peridan/PINN1/e/PIN-24


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | lstm      | LSTM    | 207 K  | train
1 | fc        | Linear  | 1.2 K  | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
209 K     Trainable params
0         Non-trainable params
209 K     Total params
0.836     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 15.61it/s]

c:\Users\ADMIN\anaconda3\envs\PinnProject\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

Epoch 0: 100%|██████████| 244/244 [00:05<00:00, 45.49it/s, v_num=N-24]Validation Epoch End: Loss: 1.0490841 MSE: [0.7709667  2.375589   0.6587878  0.87719953 1.0060437  0.7479808
 1.1312435  0.8183123  1.0556326 ] MAE: [0.6877934 1.3515825 0.6948146 0.7462994 0.5602547 0.6892772 0.9693329
 0.5704604 0.770847 ] R²: -0.04643934329710672
Epoch 0: 100%|██████████| 244/244 [00:10<00:00, 23.96it/s, v_num=N-24]Training Epoch End: Loss: 0.6309842 MSE: [0.14942607 0.16545261 0.16151041 0.88579595 1.0253278  0.53970027
 0.75044584 0.9859783  1.0152224 ] MAE: [0.15629618 0.2258245  0.21474294 0.45924625 0.56418025 0.46565542
 0.5712329  0.40513313 0.58509964] R²: 0.34017969415469484
Epoch 1: 100%|██████████| 244/244 [00:05<00:00, 43.32it/s, v_num=N-24]Validation Epoch End: Loss: 0.8241185 MSE: [0.59439784 2.1540227  0.50992393 0.73328    0.6484522  0.46054134
 0.42461327 0.8026435  1.089191  ] MAE: [0.58480614 1.2830303  0.6176548  0.6517588  0.49148643 0.50168467
 0.53198534 0.5439022  0.8382989

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 244/244 [00:11<00:00, 20.42it/s, v_num=N-24]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\ADMIN\anaconda3\envs\PinnProject\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 244/244 [00:06<00:00, 39.46it/s]Test Epoch End: Loss: 0.14406982 MSE: [0.0831078  0.09268803 0.07167563 0.23067963 0.2752669  0.11534791
 0.15004247 0.18228169 0.09553844] MAE: [0.18642491 0.23695542 0.18094088 0.3657271  0.3260155  0.22495574
 0.26355052 0.2976932  0.24105343] R²: 0.8399912555252496
Testing DataLoader 0: 100%|██████████| 244/244 [00:06<00:00, 39.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test/batch/mae_alt       0.18095342814922333
  test/batch/mae_hding      0.22509358823299408
   test/batch/mae_lat       0.18650038540363312
   test/batch/mae_lon       0.2370768040418625
  test/batch/mae_pitch      0.3659030497074127
   test/batch/mae_roll      0.3261619508266449
    test/batch/m

In [37]:
baseline_model = BaselineModel()

# Initialize the trainer
trainer = pl.Trainer(max_epochs=10)

# Train the model
trainer.fit(baseline_model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ValueError: optimizer got an empty parameter list